In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

#Import our helper functions
import helper_functions

# Step 1: View the list of reports available

In [2]:
#View the list of precheck reports
report_lst = helper_functions.get_available_precheck_reports()

report_lst

,ReportName,ReportPath
61,Pre-Check Deduction,/content/folder[@name='zzzCompany Folders']/fo...
62,Pre-Check Earnings,/content/folder[@name='zzzCompany Folders']/fo...
63,Pre-Check Master,/content/folder[@name='zzzCompany Folders']/fo...
64,Pre-Check Payroll Register,/content/folder[@name='zzzCompany Folders']/fo...
65,Pre-Check Tax,/content/folder[@name='zzzCompany Folders']/fo...
66,Prenote,/content/folder[@name='zzzCompany Folders']/fo...


In [3]:
report_path = report_lst[report_lst['ReportName'] == 'Pre-Check Deduction']['ReportPath'].loc[61]

## Step 2: Retrieve the report and export as a CSV

In [4]:
df = helper_functions.retrieve_report(report_path) 
df 

c:\UKG\Reports as a service\helper_functions.py:367: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('{', '')
c:\UKG\Reports as a service\helper_functions.py:368: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('}', '')
c:\UKG\Reports as a service\helper_functions.py:369: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('[', '')
c:\UKG\Reports as a service\helper_functions.py:370: FutureWarning: The defa

,period_control_date,employee_number,deduction/benefit_code,deduction/benefit_(stub),current_amount_(employee),current_amount_(employer)
0,2022-12-09T00:00:00,001008339,OFFC,Off-Cycle Check,-1928.71,0
1,2022-12-09T00:00:00,001008584,OFFC,Off-Cycle Check,-344.75,0
2,2022-12-09T00:00:00,001008586,OFFC,Off-Cycle Check,-693.27,0


In [5]:
#Remove the T's from the column names
df['period_control_date'] = df['period_control_date'].str.replace('T', ' ')

df

,period_control_date,employee_number,deduction/benefit_code,deduction/benefit_(stub),current_amount_(employee),current_amount_(employer)
0,2022-12-09 00:00:00,001008339,OFFC,Off-Cycle Check,-1928.71,0
1,2022-12-09 00:00:00,001008584,OFFC,Off-Cycle Check,-344.75,0
2,2022-12-09 00:00:00,001008586,OFFC,Off-Cycle Check,-693.27,0


# Step 3: Export the report as a CSV

In [6]:
df.to_csv('ukg_bulk_insert.csv', index=False)

# Step 4: Insert the report into BiData2 as a table

In [7]:
helper_functions.create_sql_table_v2('STG_PreCheck_Deduction')

# Step 5: Close the API connection

In [8]:
helper_functions.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'